# Point Processing (Tutorial 1)
***
# Table of Contents
1.   [Imports](#Imports)
2.   [Exercise 1](#Exercise-1)
3.   [Exercise 2 - Thresholding](#Exercise-2---Thresholding)
4.   [Exercise 3 - Histogram](#Exercise-3---Histogram)
5.   [Exercise 4 - Power Transform](#Exercise-4---Power-Transform)
6.   [Exercise 5 - Bit Plane Slicing](#Exercise-5---Bit-Plane-Slicing)
7.   [Exercise 6 - Application to Video](#Exercise-6---Application-to-Video)

# Imports

Only 3 libraries are needed for this project:
* opencv (cv2) - For image processing
* numpy - For its arrays
* matplotlib - Plotting histograms

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Exercise 1

### Code Explanation

For this exercise I load the image into the original_image variable. The next step is to take the
width and height of the image, this can be done via  shape which returns the dimensions of a given tensor.
The channels of the image are ignored.

The image is then converted into a gray scale version of itself to reduce colour complexity for other metrics.

In [2]:
original_image = cv2.imread("cursed.jpg")
height, width, _ = original_image.shape
gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

### cursed.jpg

The image I will be using for this lab is one of my failed attempts of creating a pie chart for one of my statistics units
from last year. It features a sun like object in the centre with black lines coming out from the centre, it is coloured in
full RGB while the background is gray.

<img src="cursed.jpg">

### Code Explanation

Here I loop for the number of times an axis needs to be split, this can be changed by changing the h_split/w_split
variables.

In the loop:

* x, y are the "starting" pixel coordinates for the new split.
* h, w are the new height and width for the new splits.

img2 is then initialised as a slice (Python list slicing) of original_image.
``` Python
[y:y+h, x:x+w]
```
Can be seen as:

For the **width** of **img** start at **y till(:) y+h** and similarly, for the **height** of **img** start at
**x till(:) x+w**

The last 4 lines deal with saving the new split image as its own jpg file as well as building the returning dict.
A count is kept to distinguish clearly between the splits.

images is a dictionary that will be used to avoid creating/loading the quadrants multiple times.

In [3]:
def splitImage(input, w_split=2, h_split=2, detail="Unedited"):
    height, width = input.shape
    count = 0
    images = {}

    for ih in range(h_split):
        for iw in range(w_split):
            x = width // w_split * iw
            y = height // h_split * ih
            h = height // h_split
            w = width // w_split
            img2 = input[y:y+h, x:x+w]

            NAME = "Outputs/"+ detail + "/Quadrant " + str(count)
            cv2.imwrite(NAME +  ".png", img2, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            images["Quadrant " + str(count)] = {'img': img2, 'dark':False}
            count += 1

    return images

### Code Explanation

I use the splitImage function. Due to the generated nature of the image being used, the quadrants end up being very
similar to each other.

In [4]:
images = splitImage(gray)

### Results

Quadrants | .
-|- 
<img src="./Outputs/Unedited/Quadrant 0.png" width="240" height="135"> | <img src="./Outputs/Unedited/Quadrant 1.png" width="240" height="135">
<img src="./Outputs/Unedited/Quadrant 2.png" width="240" height="135"> | <img src="./Outputs/Unedited/Quadrant 3.png" width="240" height="135">



# Exercise 2 - Thresholding

### Code Explanation

The isDark function returns whether a given image is dark or not. The darkness sensitivity can be changed by giving the
alpha value, which by default is 0.5, i.e. 50%.

The function is very simple in itself;
* Each pixel is checked and if the channel values of the bgr values are < 256*alpha then it is considered a
dark pixel.
* This is done by utilizing numpy's < operator which returns a bool ndarray where the condition is either true or false.
Then using numpy's count_nonzero I can get a count of all the instances where the pixel is dark.

This method works much faster than looping through all the pixels in 2 for loops like so:

```Python
for x in range(h):
    for y in range(w):
        if np.less(image[x,y], T):
            dark += 1
```

In [5]:
def isDark(image, alpha=0.5):
    h, w = image.shape
    T = np.array([255*alpha])
    dark = np.count_nonzero(image < T)
    return dark/(h*w) > 0.5

### Code Explanation

Here I loop through each image generated previously and check whether it is dark or not.

In [6]:
for img_name in images:
    images[img_name]['dark'] = isDark(images[img_name]['img'])

# Exercise 3 - Histogram

### Code Explanation

Each image has its histogram calculated using the opencv function. The parameters are passed like this:

* images
    * The image from the images dict passed to the generateHistograms function
* channels
    * 0, since the images are grayscale, if the images where coloured i would have to create a histogram for each colour (1,2,3)
* mask
    * no mask is given
* histsize
    * The maximum colour value is 255
* ranges
    * colour ranges from 0 to 255

Each plot has its x and y axes labelled, as well as having a title which indicates whether the image it is based on is dark
or not.

Instead of displaying the histograms when generated they are saved as a png file and the function returns a list of their
locations. Which can later be used to display the histograms.

plt.clf is used to clear previously loaded/generated plots, especially useful when creating histograms for the video.

In [7]:
%%capture
def generateHistograms(images, detail="Unedited"):
    img_names = []
    for img_name in images:
        hist = cv2.calcHist([images[img_name]['img']], [0], None, [255], [0, 255])
        plt.ylabel('Pixels')
        plt.xlabel('Intensity')
        plt.title(img_name + ', is Dark?: ' + str(images[img_name]['dark']))
        plt.plot(hist)
        plt.xlim([0, 256])
        plt.savefig('Histograms/' + detail + '/'+ img_name + ' histogram.png')
        img_names.append('Histograms/' + detail + '/'+ img_name + ' histogram.png')
        plt.clf()
    return img_names
generateHistograms(images)

### Result Discussion
Since the main feature of cursed.jpg is split between the 4 quadrants the majority of these quadrants is now made of what
was once the background. Due to the auto-generated nature of the image, each quadrant's colour histogram displays a spike
for pixels with (127) as their value. I.e cursed.jpg in grayscale is a very medium level gray.

Histograms | .
-|-
<img src="./Histograms/Unedited/Quadrant 0 histogram.png"> | <img src="./Histograms/Unedited/Quadrant 1 histogram.png" >
<img src="./Histograms/Unedited/Quadrant 2 histogram.png"> | <img src="./Histograms/Unedited/Quadrant 3 histogram.png">

# Exercise 4 - Power Transform

### Code Explanation

First I copy image to _cpy, then I apply the power transform using numpy given gamma. The larger gamma the darker the
image will become and vice versa. 1 being neutral.

In [8]:
def powerTransform(image, gamma=2):
    _cpy = image.copy()
    _cpy = np.array(255*(_cpy/255)**gamma,dtype='uint8')
    return _cpy

### Code Explanation

Here I loop through each image generated previously and apply the power transform, as well as more generate histograms.

In [9]:
%%capture
transformed_images = {}
count = 0
for img_name in images:
    transformed = powerTransform(images[img_name]['img'])
    name = "Outputs/Transformed/Quadrant " + str(count)
    cv2.imwrite(name +  ".png", transformed, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    transformed_images[img_name] = {'img':  transformed,
                                    'dark': isDark(transformed)}
    count += 1

generateHistograms(transformed_images, "Transformed")

### Result Discussion

The images are applied with the base gamma of 2 and are visibly darker, the histograms also show this shift to the left in pixel values. The smaller value a pixel has the darker its gray is.

Quadrants | .
-|- 
<img src="./Outputs/Transformed/Quadrant 0.png" width="240" height="135"> | <img src="./Outputs/Transformed/Quadrant 1.png" width="240" height="135">
<img src="./Outputs/Transformed/Quadrant 2.png" width="240" height="135"> | <img src="./Outputs/Transformed/Quadrant 3.png" width="240" height="135">

Histograms | .
-|-
<img src="./Histograms/Transformed/Quadrant 0 histogram.png"> | <img src="./Histograms/Transformed/Quadrant 1 histogram.png" >
<img src="./Histograms/Transformed/Quadrant 2 histogram.png"> | <img src="./Histograms/Transformed/Quadrant 3 histogram.png">

# Exercise 5 - Bit Plane Slicing

### Code Explanation

With np.full I create a copy of the inputted image and fills it with 2^bit. This creates the bit plane of given image

In [10]:
def Slice(image, bit):
    plane = np.full((image.shape[0], image.shape[1]), 2 ** bit, np.uint8)
    res = cv2.bitwise_and(plane, image)
    x = res * 255
    return x


### Code Explanation

Here I loop through each image generated previously and apply the bit slice twice, for 2 and 7. When this process is
finished histograms are generated for comparison.

In [11]:
%%capture
Sliced_images = {}
slices = [2,7]

for slice in slices:
    count = 0
    for img_name in images:
            sliced = Slice(images[img_name]['img'], slice)
            name = "Outputs/Sliced/Sliced Quadrant " + str(slice) + str(count)
            cv2.imwrite(name +  ".png", sliced, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            Sliced_images[name[16:]] = {'img':  sliced,
                                             'dark': isDark(sliced)}
            count += 1
generateHistograms(Sliced_images, "Sliced")

### Result Discussion

For this exercise I sliced at the 3rd and 8th bit, the bit count starts at 0. Both have different results. The lower bit slicing features the more denseley categorised areas of the pie chart while the higher bit slice features what would have been the labels as well as the less dense areas of the pie chart.


The way both are featuring is different since for the lower slice I am looking at the black parts whereas for the higher bit slice I am looking at the white parts.

Unfortunately for the higher bit slice some noise is being generated even though I tried to save in lossless format.

Quadrants | Spliced at 2
-|- 
<img src="./Outputs/Sliced/Sliced Quadrant 20.png" width="240" height="135"> | <img src="./Outputs/Sliced/Sliced Quadrant 21.png" width="240" height="135">
<img src="./Outputs/Sliced/Sliced Quadrant 22.png" width="240" height="135"> | <img src="./Outputs/Sliced/Sliced Quadrant 23.png" width="240" height="135">

Histograms | Spliced at 2
-|-
<img src="./Histograms/Sliced/Sliced Quadrant 20 histogram.png"> | <img src="./Histograms/Sliced/Sliced Quadrant 21 histogram.png" >
<img src="./Histograms/Sliced/Sliced Quadrant 22 histogram.png"> | <img src="./Histograms/Sliced/Sliced Quadrant 23 histogram.png">

Quadrants | Spliced at 7
-|- 
<img src="./Outputs/Sliced/Sliced Quadrant 70.png" width="240" height="135"> | <img src="./Outputs/Sliced/Sliced Quadrant 71.png" width="240" height="135">
<img src="./Outputs/Sliced/Sliced Quadrant 72.png" width="240" height="135"> | <img src="./Outputs/Sliced/Sliced Quadrant 73.png" width="240" height="135">


While hard to notice the graph leans at 0

Histograms | Spliced at 7
-|-
<img src="./Histograms/Sliced/Sliced Quadrant 70 histogram.png"> | <img src="./Histograms/Sliced/Sliced Quadrant 71 histogram.png" >
<img src="./Histograms/Sliced/Sliced Quadrant 72 histogram.png"> | <img src="./Histograms/Sliced/Sliced Quadrant 73 histogram.png">


# Exercise 6 - Application to Video

### League.mp4

The video I will be using for this exercise is the only mp4 video I found on my pc, and since I don't have a webcam I'm
stuck with it. It features some gameplay footage from the game League of Legends. The unprocessed version is 1920x1080 (FHD)
with full RGB. When turned to grayscale it features a more varied range of grays than cursed.jpg.

<img src="Screenshot.jpg">


### Code Explanation

In this first part I load the video using cv2.VideoCapture.

I get the fps of the video and divided it by 4 so that I only process 4 frames each second. Any more and the loop would
just lag.

Using the **read()** function I get ret and the frame. If the video stops, ret is false. This will make it easier for me
to stop the process and program. Unfortunately I was not able to create an interrupt that stops the process mid video.

In [12]:
vidFile = cv2.VideoCapture('League.mp4')
fps = vidFile.get(cv2.CAP_PROP_FPS)
print(fps)
ret, frame = vidFile.read()
count = 0

35.154063403335364


A count is incremented each frame, if this count reaches 8 I process the frame.

Similar steps to previous exercises are done here, namely:
* Converting to grayscale
* Splitting the image
* Checking if the quadrants are dark
* Applying the power transform if the quadrant is dark
* Generating histograms

The differences here are:
* That I resize the frame to 800x450 (retaining 16:9)
* Merging the quadrants and generated histograms into 2 windows using numpy concatenations
    * Also the return from **generatedHistograms()** is used to load the images back into memory

**cv2.waitKey(int(1/fps*1000))** is used as a small wait so that each frame can be loaded.

In [13]:
while ret:
    if count % 8 == 0:
        frame = cv2.resize(frame, (800, 450))
        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        quadrants = splitImage(grayFrame,w_split=2,h_split=2,detail="Video")

        for img_name in quadrants:
            quadrants[img_name]['dark'] = isDark(quadrants[img_name]['img'], alpha=0.25)
            if quadrants[img_name]['dark']:
                quadrants[img_name]['img'] = powerTransform(quadrants[img_name]['img'], gamma=0.3)

        histogram_names = generateHistograms(quadrants, "Video")
        histograms = []
        for histogram in histogram_names:
            histograms.append(cv2.imread(histogram))

        histogramStitchedH1 = np.concatenate((histograms[0], histograms[1]),axis=1)
        histogramStitchedH2 = np.concatenate((histograms[2], histograms[3]),axis=1)
        histogramStitchedF = np.concatenate((histogramStitchedH1, histogramStitchedH2),axis=0)
        cv2.imshow("Histograms", histogramStitchedF)

        frameStitchedH1 = np.concatenate((quadrants['Quadrant 0']['img'],quadrants['Quadrant 1']['img']),axis=1)
        frameStitchedH2 = np.concatenate((quadrants['Quadrant 2']['img'],quadrants['Quadrant 3']['img']),axis=1)
        frameStitchedF = np.concatenate((frameStitchedH1, frameStitchedH2),axis=0)
        cv2.imshow("Processed Video", frameStitchedF)

        count = 0

    cv2.waitKey(int(1/fps*1000))
    count += 1
    ret, frame = vidFile.read()
vidFile.release()
cv2.destroyAllWindows()

<Figure size 432x288 with 0 Axes>